In [ ]:
import keras
from numpy import *
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, SimpleRNN,Conv1D, AveragePooling1D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from keras.layers import Input
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix,f1_score,recall_score,accuracy_score,precision_score,roc_auc_score,precision_recall_curve
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# install to see the progress bar
#!pip install tqdm

In [ ]:
#read file after train/test split  
x_train=pd.read_csv('xtrain.csv')
y_train=pd.read_csv('ytrain.csv')
x_test=pd.read_csv('xtest.csv')
y_test=pd.read_csv('ytest.csv')

In [ ]:
#Dynamic features
#PAY(X6-X11)
pay_train = x_train[['X6','X7','X8','X9','X10','X11']]
pay_test = x_test[['X6','X7','X8','X9','X10','X11']]
#BILL_AMT(X12-X17)
bill_amt_train = x_train[['X12','X13','X14','X15','X16','X17']]
bill_amt_test = x_test[['X12','X13','X14','X15','X16','X17']]
#PAY_AMT(X18-X23)
pay_amt_train = x_train[['X18','X19','X20','X21','X22','X23']]
pay_amt_test = x_test[['X18','X19','X20','X21','X22','X23']]
#PAY
pay_train = pay_train[pay_train.columns[::-1]] #Since pay0~pay6 are data from Sept. to Apr., so we'll have to feed in the data backwards to that it's in chronological order
pay_test = pay_test[pay_test.columns[::-1]]
#BILL_AMT
bill_amt_train = bill_amt_train[bill_amt_train.columns[::-1]]
bill_amt_test = bill_amt_test[bill_amt_test.columns[::-1]]
#PAY_AMT
pay_amt_train = pay_amt_train[pay_amt_train.columns[::-1]]
pay_amt_test = pay_amt_test[pay_amt_test.columns[::-1]]
#reshape train data
#number of features
nb_features=1
#number of observing timepoints
nb_times=6
pay_train_time = np.zeros((len(pay_train), nb_times, nb_features*1))
bill_amt_train_time = np.zeros((len(bill_amt_train), nb_times, nb_features*1))
pay_amt_train_time = np.zeros((len(pay_amt_train), nb_times, nb_features*1))
for i in range(0, nb_times):
    pay_train_time[:,i,:] = pay_train.values[:, nb_features*i:nb_features*(i+1)]
    bill_amt_train_time[:,i,:] = bill_amt_train.values[:, nb_features*i:nb_features*(i+1)]
    pay_amt_train_time[:,i,:] = pay_amt_train.values[:, nb_features*i:nb_features*(i+1)]
pay_test_time = np.zeros((len(pay_test), nb_times, nb_features*1)) 
bill_amt_test_time = np.zeros((len(bill_amt_test), nb_times, nb_features*1))
pay_amt_test_time = np.zeros((len(pay_amt_test), nb_times, nb_features*1))
for i in range(0, nb_times):
    pay_test_time[:,i,:] = pay_test.values[:, nb_features*i:nb_features*(i+1)]
    bill_amt_test_time[:,i,:] = bill_amt_test.values[:, nb_features*i:nb_features*(i+1)]
    pay_amt_test_time[:,i,:] = pay_amt_test.values[:, nb_features*i:nb_features*(i+1)]
#print(pay_train_time.shape) #checking the results

In [ ]:
type( pay_test)

pandas.core.frame.DataFrame

In [ ]:
pay_train_ar=pd.DataFrame(pay_train).to_numpy()
pay_test_ar=pd.DataFrame(pay_test).to_numpy()

### CNN

In [ ]:
def CNN(layer,neuron,fun):
  model=Sequential()
  # one layer CNN
  model.add(Conv1D(neuron, 2, input_shape=(6,1))) # 2 means kernal size
  model.add(BatchNormalization())
  model.add(Activation(fun))
  # two layer CNN
  if layer == 2 or 3:
    model.add(Conv1D(neuron, 2))
    model.add(BatchNormalization())
    model.add(Activation(fun))  
  # three layer CNN
  if layer == 3:
    model.add(Conv1D(neuron, 2))
    model.add(BatchNormalization())
    model.add(Activation(fun)) 
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid', name='output'))

  print(model.summary())

  callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)]
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['binary_accuracy'])
  history1 = model.fit(pay_train_ar, y_train,validation_split=0.2,epochs = 50, batch_size = 512, callbacks = callbacks,verbose=0)
  history2 = model.fit(bill_amt_train_time, y_train,validation_split=0.2,epochs = 50, batch_size = 512, callbacks = callbacks,verbose=0)
  history3 = model.fit(pay_amt_train_time, y_train,validation_split=0.2,epochs = 50, batch_size = 512, callbacks = callbacks,verbose=0)
  layer_name='output'
  intermediate_layer_model=Model(inputs=model.input,outputs=(model.get_layer(layer_name).output))
  # pay
  intermediate_output=intermediate_layer_model.predict([pay_train_ar])
  intermediate_output=pd.DataFrame(intermediate_output)
  new1_train=pd.concat([x_train, intermediate_output], axis=1)
  new1_train.rename(columns={0:'TSEF_PAY'}, inplace = True)

  intermediate_output=intermediate_layer_model.predict([pay_test_ar])
  intermediate_output=pd.DataFrame(intermediate_output)
  new1_test=pd.concat([x_test, intermediate_output], axis=1)
  new1_test.rename(columns={0:'TSEF_PAY'}, inplace = True)
  #bill_amt
  intermediate_output=intermediate_layer_model.predict([bill_amt_train_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new2_train=pd.concat([new1_train, intermediate_output], axis=1)
  new2_train.rename(columns={0:'TSEF_BILL_AMT'}, inplace = True)

  intermediate_output=intermediate_layer_model.predict([bill_amt_test_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new2_test=pd.concat([new1_test, intermediate_output], axis=1)
  new2_test.rename(columns={0:'TSEF_BILL_AMT'}, inplace = True)
  #pay_amt
  intermediate_output=intermediate_layer_model.predict([pay_amt_train_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new3_train=pd.concat([new2_train, intermediate_output], axis=1)
  new3_train.rename(columns={0:'TSEF_PAY_AMT'}, inplace = True)

  intermediate_output=intermediate_layer_model.predict([pay_amt_test_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new3_test=pd.concat([new2_test, intermediate_output], axis=1)
  new3_test.rename(columns={0:'TSEF_PAY_AMT'}, inplace = True)

  return new3_train,new3_test

### RNN

In [ ]:
def RNN(layer,neuron,fun):
  # one layer RNN
  if layer ==1:
    model=Sequential()
    model.add(SimpleRNN(neuron, return_sequences=False, input_shape=(nb_times, nb_features*1),unroll=True)) 
    model.add(BatchNormalization())
    model.add(Activation(fun))
  # two layer RNN
  if layer == 2:
    model=Sequential()
    model.add(SimpleRNN(neuron, return_sequences=True, input_shape=(nb_times, nb_features*1),unroll=True)) 
    model.add(BatchNormalization())
    model.add(Activation(fun))
    model.add(SimpleRNN(neuron,return_sequences=False,unroll=True)) 
    model.add(BatchNormalization())
    model.add(Activation(fun))
  # three layer RNN
  if layer == 3:
    model=Sequential()
    model.add(SimpleRNN(neuron, return_sequences=True, input_shape=(nb_times, nb_features*1),unroll=True)) 
    model.add(BatchNormalization())
    model.add(Activation(fun))
    model.add(SimpleRNN(neuron,return_sequences=True,unroll=True)) 
    model.add(BatchNormalization())
    model.add(SimpleRNN(neuron,return_sequences=False ,unroll=True))
    model.add(BatchNormalization())
    model.add(Activation(fun))
   
  model.add(Dense(1, activation='sigmoid', name='output'))
  callbacks=[EarlyStopping(monitor = 'val_loss', patience=30)]
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['binary_accuracy'])
  history1=model.fit(pay_train_time, y_train, epochs=50, validation_data=(pay_test_time, y_test), batch_size=512, callbacks=callbacks,verbose=0)
  history2=model.fit(bill_amt_train_time, y_train, epochs=50, validation_data=(bill_amt_test_time, y_test), batch_size=512, callbacks=callbacks,verbose=0)
  history3=model.fit(pay_amt_train_time, y_train, epochs=50, validation_data=(pay_amt_test_time, y_test), batch_size=512, callbacks=callbacks,verbose =0)
  
  layer_name='output'
  intermediate_layer_model=Model(inputs=model.input,outputs=(model.get_layer(layer_name).output))
  # pay
  intermediate_output=intermediate_layer_model.predict([pay_train_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new1_train=pd.concat([x_train, intermediate_output], axis=1)
  new1_train.rename(columns={0:'TSEF_PAY'}, inplace = True)

  intermediate_output=intermediate_layer_model.predict([pay_test_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new1_test=pd.concat([x_test, intermediate_output], axis=1)
  new1_test.rename(columns={0:'TSEF_PAY'}, inplace = True)
  # bill_amt
  intermediate_output=intermediate_layer_model.predict([bill_amt_train_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new2_train=pd.concat([new1_train, intermediate_output], axis=1)
  new2_train.rename(columns={0:'TSEF_BILL_AMT'}, inplace = True)

  intermediate_output=intermediate_layer_model.predict([bill_amt_test_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new2_test=pd.concat([new1_test, intermediate_output], axis=1)
  new2_test.rename(columns={0:'TSEF_BILL_AMT'}, inplace = True)
  # pay_amt
  intermediate_output=intermediate_layer_model.predict([pay_amt_train_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new3_train=pd.concat([new2_train, intermediate_output], axis=1)
  new3_train.rename(columns={0:'TSEF_PAY_AMT'}, inplace = True)
  
  intermediate_output=intermediate_layer_model.predict([pay_amt_test_time])
  intermediate_output=pd.DataFrame(intermediate_output)
  new3_test=pd.concat([new2_test, intermediate_output], axis=1)
  new3_test.rename(columns={0:'TSEF_PAY_AMT'}, inplace = True)

  return new3_train,new3_test

### LR 

In [ ]:
def logistic(x_newtrain,x_newtest,result,fun,layer,neuron):
  logreg = LogisticRegression( ).fit(x_newtrain, y_train)
  logreg1 = LogisticRegression( ).fit(x_newtest, y_test)
  y_prob = logreg.predict_proba(x_newtrain)[:,1]
  y_prob1 = logreg.predict_proba(x_newtest)[:,1]
  y_pred = logreg.predict(x_newtrain)
  y_pred1 = logreg.predict(x_newtest)
  # get the threshold of f1-score
  best_thr1,f1_score_train= return_best_thr(y_train,y_prob)
  best_thr2,f1_score_test= return_best_thr(y_test,y_prob1)
  # get csv data form
  new = pd.DataFrame.from_dict( {"model": ["LR"],
                               "FE_activation":[fun],
                               "FE_layer":[str(layer)],
                               "FE_neuron":[str(neuron)],
                               "data":[data],
                               "train_accuracy":[accuracy_score(y_train,y_pred)],
                               "train_precision":[precision_score(y_train,y_pred)],
                               "train_recall":[recall_score(y_train,y_pred)],
                               "train_F1_score":[f1_score_train],
                               "train_AUC":[roc_auc_score(y_train, y_pred)],
                               "test_accuracy":[accuracy_score(y_test, y_pred1)],
                               "test_precision":[precision_score(y_test,y_pred1)],
                               "test_recall":[recall_score(y_test,y_pred1)],
                               "test_F1_score":[f1_score_test],
                               "test_AUC":[roc_auc_score(y_test,y_pred1)]}) 
  result = result.append(new,ignore_index=True) 
  return result

## DT

In [ ]:
def DT(x_train,x_test,result,fun,layer,neuron):
  for depth in tqdm(range(5,10,1)):
    classifier = DecisionTreeClassifier(random_state=0, max_depth=depth)
    classifier.fit(x_train, y_train)
    
    y_pred=classifier.predict(x_train)
    y_prob=classifier.predict_proba(x_train)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_prob, pos_label=1)
    AUC =round(auc(fpr, tpr),4)
    y_pred1=classifier.predict(x_test)
    y_prob1=classifier.predict_proba(x_test)[:,1]
    fpr1, tpr1, thresholds1 = metrics.roc_curve(y_test, y_prob1, pos_label=1)
    AUC1 =round(auc(fpr1, tpr1),4)
    # get the threshold of f1-score
    best_thr1,f1_score_train= return_best_thr(y_train,y_prob)
    best_thr2,f1_score_test= return_best_thr(y_test,y_prob1)
    # get csv data form
    new = pd.DataFrame.from_dict( {"model": ["DT"],
                                "FE_activation":[fun],
                                "FE_layer":[str(layer)],
                                "FE_neuron":[str(neuron)],
                                "max_depth":[str(depth)],             
                                "data":[data],
                                "train_accuracy":[metrics.accuracy_score(y_train, y_pred)],
                                "train_precision":[metrics.precision_score(y_train, y_pred)],
                                "train_recall":[metrics.recall_score(y_train, y_pred)],
                                "train_F1_score":[f1_score_train],
                                "train_AUC":[str(AUC)],
                                "test_accuracy":[metrics.accuracy_score(y_test, y_pred1)],
                                "test_precision":[metrics.precision_score(y_test, y_pred1)],
                                "test_recall":[metrics.recall_score(y_test, y_pred1)],
                                "test_F1_score":[f1_score_test],
                                "test_AUC":[str(AUC1)]}) 
    result = result.append(new,ignore_index=True) 
  return result

## LGBM

In [ ]:
def LGBM(x_train,x_test,result,fun,layer,neuron):
  leaves = [28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
  l=0
  for depth in tqdm(range (5,8,1)):
    for count in range (5):
      classifier= lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=leaves[l], max_depth=depth, learning_rate=0.5, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.05, min_child_weight=0.9, min_child_samples=20, subsample=1.0, 
                                    subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, silent=True, importance_type='split')
      classifier.fit(x_train, y_train)
      y_pred=classifier.predict(x_train)
      y_prob=classifier.predict_proba(x_train)[:,1]
      fpr, tpr, thresholds = metrics.roc_curve(y_train, y_prob, pos_label=1)
      AUC =round(auc(fpr, tpr),4)
      y_pred1=classifier.predict(x_test)
      y_prob1=classifier.predict_proba(x_test)[:,1]
      fpr1, tpr1, thresholds1 = metrics.roc_curve(y_test, y_prob1, pos_label=1)
      AUC1 =round(auc(fpr1, tpr1),4)
      # get the threshold of f1-score
      best_thr1,f1_score_train= return_best_thr(y_train,y_prob)
      best_thr2,f1_score_test= return_best_thr(y_test,y_prob1)
      # get csv data form
      new = pd.DataFrame.from_dict( {"model": ["LGBM"],
                                    "FE_activation":[fun],
                                    "FE_layer":[str(layer)],
                                    "FE_neuron":[str(neuron)],
                                    "max_depth":[str(depth)],
                                    "num_leaves":[str(leaves[l])],          
                                    "data":[data],
                                    "train_accuracy":[metrics.accuracy_score(y_train, y_pred)],
                                    "train_precision":[metrics.precision_score(y_train, y_pred)],
                                    "train_recall":[metrics.recall_score(y_train, y_pred)],
                                    "train_F1_score":[f1_score_train],
                                    "train_AUC":[str(AUC)],
                                    "test_accuracy":[metrics.accuracy_score(y_test, y_pred1)],
                                    "test_precision":[metrics.precision_score(y_test, y_pred1)],
                                    "test_recall":[metrics.recall_score(y_test, y_pred1)],
                                    "test_F1_score":[f1_score_test],
                                    "test_AUC":[str(AUC1)]}) 
      
      result = result.append(new,ignore_index=True) 
      l=l+1
  return result

## NN 

In [ ]:
def NN(x_train,x_test,nn_layer,result,fun,layer,neuron,d,function,data):
  sequence = [0.1,0.2,0.3,0.4,0.5]
  for l in sequence:
    # NN layer 1 
    model = Sequential()
    model.add(Dense(128,input_dim=d))
    model.add(BatchNormalization())
    model.add(Activation(function))
    model.add(Dropout(l))
    # NN layer 2
    if nn_layer == 2 or 3:
      model.add(Dense(128))
      model.add(BatchNormalization())
      model.add(Activation(function))
      model.add(Dropout(l))
    # NN layer 3
    if nn_layer ==3:
      model.add(Dense(128))
      model.add(BatchNormalization())
      model.add(Activation(function))
      model.add(Dropout(l))
    model.add(Dense(1, name='output'))
    model.add(Dense(1, activation='sigmoid'))
    callbacks=[EarlyStopping(monitor = 'val_loss', patience=30)]
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['binary_accuracy'])
    history=model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), batch_size=512, callbacks=callbacks,verbose=0)
    y_pred = model.predict(x_train)
    y_prob = model.predict_proba(x_train)
    # get the threshold of f1-score
    best_thr1,f1_score_train= return_best_thr(y_train,y_prob)
    for i in range(len(y_pred)):
      if y_pred[i] > best_thr1:
        y_pred[i] =  1
      else:
        y_pred[i] = 0
    y_pred = pd.DataFrame(y_pred)
    
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_prob, pos_label=1)
    AUC = round(auc(fpr, tpr),4)
    acc = metrics.accuracy_score(y_train,y_pred)
    pre = metrics.precision_score(y_train,y_pred)
    rec = metrics.recall_score(y_train,y_pred)
    F1 = 2*(pre*rec)/(pre+rec)
    y_pred1=model.predict(x_test)
    y_prob1 = model.predict_proba(x_test)
    # get the threshold of f1-score
    best_thr2,f1_score_test= return_best_thr(y_test,y_prob1)
    for i in range(len(y_pred1)):
      if y_pred1[i]> best_thr2:
        y_pred1[i] = 1
      else:
        y_pred1[i] = 0    
    
    fpr1, tpr1, thresholds1 = metrics.roc_curve(y_test, y_prob1, pos_label=1)
    AUC1 = round(auc(fpr1, tpr1),4)
    acc1 = metrics.accuracy_score(y_test,y_pred1)
    pre1 = metrics.precision_score(y_test,y_pred1)
    rec1 = metrics.recall_score(y_test,y_pred1)
    F11 = 2*(pre1*rec1)/(pre1+rec1) 
    # get csv data form
    new = pd.DataFrame.from_dict( {"model": ["NN"],
                                "FE_activation":[fun],
                                "FE_layer":[str(layer)],
                                "FE_neuron":[str(neuron)],
                                "NN_layer":[str(nn_layer)],
                                "Dropout":[str(l)],
                                "nn activation function":[function],            
                                "data":[data],
                                "train_accuracy":[acc],
                                "train_precision":[pre],
                                "train_recall":[rec],
                                "train_F1_score":[F1],
                                "train_AUC":[str(AUC)],
                                "test_accuracy":[acc1],
                                "test_precision":[pre1],
                                "test_recall":[rec1],
                                "test_F1_score":[F11],
                                "test_AUC":[str(AUC1)]}) 
    result = result.append(new,ignore_index=True) 
  return result

## threshold

In [ ]:
# dynamic select F1-score thershold
def return_best_thr(y_true, y_pred):
    precs, recs, thrs = precision_recall_curve(y_true, y_pred)
    f1s = 2 * precs * recs / (precs + recs)
    f1s = f1s[:-1]
    thrs = thrs[~np.isnan(f1s)]
    f1s = f1s[~np.isnan(f1s)]
    best_thr = thrs[np.argmax(f1s)]
    f1_score = np.max(f1s)
    return best_thr,f1_score

## core function

In [ ]:
# check shape (24000,87) 
x_train.shape

(24000, 87)

In [ ]:
# Input space
data = input("Input data: XB/XTR ")
if data=="XB":
  state ="original"
else:
  state = input("Input state: CNN/ RNN ")
  fun = input("Input CNN or RNN activation function: relu/ tanh ")
model = input("Input model: LR/DT/LGBM/NN ")
if model == "NN":
  nn_fun= input("Input NN activaiton function: relu/tanh ")

Input data: XB/XTR XTR
Input state: CNN/ RNN CNN
Input CNN or RNN activation function: relu/ tanh relu
Input model: LR/DT/LGBM/NN LR


In [ ]:
# main function
result = pd.DataFrame(columns=['model',"FE_activation","FE_layer","FE_neuron",'data','train_accuracy','train_precision','train_recall','train_F1_score','train_AUC',
                               'test_accuracy','test_precision','test_recall','test_F1_score','test_AUC'])
start = time.clock()
if state =="original": #XB
  fun =""
  layer=""
  neuron=""
  if model =="LR":
    result =logistic(x_train,x_test,result,fun,layer,neuron)
  if model =="DT":
    result = DT(x_train,x_test,result,fun,layer,neuron)
  if model =="LGBM":
    result = LGBM(x_train,x_test,result,fun,layer,neuron)
  if model =="NN":
    for nn_layer in range (1,4):
      fun =""
      layer=""
      neuron=""
      data="XB"
      d = 87
      result = NN(x_train,x_test,nn_layer,result,fun,layer,neuron,d,nn_fun,data)
    
elif state =="CNN": #CNN
  for layer in tqdm(range (1,4)):
    neuron =1
    while neuron !=256:
      new3_train,new3_test = CNN(layer,neuron,fun)      
      
      if data=="XTR":
        x_newtrain=new3_train
        x_newtest= new3_test
        if model=="LR":
          result = logistic(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="DT":
          result = DT(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="LGBM":
          result = LGBM(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="NN":
          d=90
          for nn_layer in range (1,4):
            x_newtrain=new3_train
            x_newtest= new3_test
            result = NN(x_newtrain,x_newtest,nn_layer,result,fun,layer,neuron,d,nn_fun,data)
      
      neuron = neuron*2
     
elif state =="RNN": #RNN  
  for layer in tqdm(range(1,4)):
    neuron =1
    while neuron !=256:
      new3_train,new3_test = RNN(layer,neuron,fun)      
      
      if data=="XTR":
        x_newtrain=new3_train
        x_newtest= new3_test
        if model=="LR":
          result = logistic(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="DT":
          result = DT(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="LGBM":
          result = LGBM(x_newtrain,x_newtest,result,fun,layer,neuron)
        if model=="NN":
          d=90
          for nn_layer in range (1,4):
            x_newtrain=new3_train
            x_newtest= new3_test
            result = NN(x_newtrain,x_newtest,nn_layer,result,fun,layer,neuron,d,nn_fun,data)
      neuron = neuron*2
end = time.clock()  
print("End, Time:",end-start)

  0%|          | 0/3 [00:00<?, ?it/s]

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 5, 1)              3         
_________________________________________________________________
batch_normalization_6 (Batch (None, 5, 1)              4         
_________________________________________________________________
activation_6 (Activation)    (None, 5, 1)              0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 4, 1)              3         
_________________________________________________________________
batch_normalization_7 (Batch (None, 4, 1)              4         
_________________________________________________________________
activation_7 (Activation)    (None, 4, 1)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4)                

ValueError: ignored

### Storing the result

In [ ]:
result

,model,FE_activation,FE_layer,FE_neuron,data,train_accuracy,train_precision,train_recall,train_F1_score,train_AUC,test_accuracy,test_precision,test_recall,test_F1_score,test_AUC,NN_layer,Dropout,nn activation function
0,NN,tanh,1,1,XTR,0.829958,0.664244,0.452055,0.537983,0.8078,0.811000,0.634279,0.421014,0.506098,0.756,1,0.1,tanh
1,NN,tanh,1,1,XTR,0.830833,0.713419,0.380327,0.496153,0.8119,0.815333,0.685286,0.364493,0.475875,0.7749,1,0.2,tanh
2,NN,tanh,1,1,XTR,0.827500,0.651795,0.455860,0.536498,0.7945,0.814167,0.637022,0.446377,0.524925,0.7657,1,0.3,tanh
3,NN,tanh,1,1,XTR,0.823792,0.689834,0.355023,0.468785,0.792,0.816333,0.700288,0.352174,0.468660,0.7705,1,0.4,tanh
4,NN,tanh,1,1,XTR,0.823083,0.689992,0.348935,0.463482,0.7901,0.817167,0.709630,0.347101,0.466180,0.7721,1,0.5,tanh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,NN,tanh,2,128,XTR,0.836958,0.765520,0.368341,0.497367,0.8377,0.807167,0.677266,0.308696,0.424092,0.7656,3,0.1,tanh
236,NN,tanh,2,128,XTR,0.830750,0.717883,0.374239,0.491996,0.8181,0.811667,0.678063,0.344928,0.457253,0.7687,3,0.2,tanh
237,NN,tanh,2,128,XTR,0.830792,0.699100,0.399163,0.508175,0.816,0.814000,0.670984,0.375362,0.481413,0.7653,3,0.3,tanh
238,NN,tanh,2,128,XTR,0.828583,0.679222,0.411720,0.512675,0.812,0.812167,0.655980,0.385507,0.485623,0.7686,3,0.4,tanh


In [ ]:
result.to_csv('XTR_RNN_tanh_NN_relu_tmp.csv', index=False)